# LLM Optimization Modelling Experiment

In [3]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [55]:
problem = '''You are a chemist in charge of a process that requires two liquids to reach the end product. Your goal is to maximize the quality of the product in the end, which is measured by the difference between the Napierian logarithm of the product of the litres used of each component and the cube root of the summation of the first component litres as well as double the second component litres. It is required that the squared difference between the litres employed of each component and the mean litres utilised of both components is below ten. This is necessary for the obtained set to be stable. Furthermore, the mixture tank available to you can only fit 80 l. Please formulate a mathematical optimization model for this problem. '''

## 2. Generate the mathematical model

In [104]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [105]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model

**Parameters:**

*  None

**Decision Variables:**

*  $x_1$: Litres of the first component used.
*  $x_2$: Litres of the second component used.

**Objective Function:**

Maximize $Z = \ln(x_1 \cdot x_2) - \sqrt[3]{x_1 + 2x_2}$ 

**Constraints:**

1. **Tank Capacity:** $x_1 + x_2 \leq 80$
2. **Stability:** $(x_1 - \frac{x_1 + x_2}{2})^2 \leq 10$
3. **Stability:** $(x_2 - \frac{x_1 + x_2}{2})^2 \leq 10$
4. **Non-negativity:** $x_1 \geq 0$
5. **Non-negativity:** $x_2 \geq 0$ 

**Explanation:**

* The objective function maximizes the quality of the end product based on the given formula.
* Constraint 1 ensures that the total volume of liquids used does not exceed the tank's capacity.
* Constraints 2 and 3 enforce the stability requirement by limiting the squared difference between each component's volume and the mean volume.
* Constraints 4 and 5 ensure that the volume of each component used is non-negative.

This model represents the problem mathematically and can be solved using appropriate optimization techniques to find the optimal volumes of each component for maximizing the product quality. 


## 3. Generate the pyomo code

In [106]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [107]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Create a concrete model
model = pyo.ConcreteModel()

# Define decision variables
model.x1 = pyo.Var(domain=pyo.NonNegativeReals)
model.x2 = pyo.Var(domain=pyo.NonNegativeReals)

# Define the objective function
model.obj = pyo.Objective(expr=pyo.log(model.x1 * model.x2) - (model.x1 + 2*model.x2)**(1/3), sense=pyo.maximize)

# Define constraints
model.tank_capacity = pyo.Constraint(expr=model.x1 + model.x2 <= 80)
model.stability1 = pyo.Constraint(expr=(model.x1 - (model.x1 + model.x2)/2)**2 <= 10)
model.stability2 = pyo.Constraint(expr=(model.x2 - (model.x1 + model.x2)/2)**2 <= 10)

# Solve the model
solver = pyo.SolverFactory('ipopt') # You can choose a different solver if you prefer
results = solver.solve(model)

# Print the results
print(results)
print("x1:", pyo.value(model.x1))
print("x2:", pyo.value(model.x2))
```

This code defines the Pyomo model for the provided mathematical optimization problem. 

**Explanation:**

1. **Import Pyomo:** Imports the Pyomo library.
2. **Create a Concrete Model:** Creates a ConcreteModel object to represent the optimization problem.
3. **Define Decision Variables:**  Defines the decision variables `x1` and `x2` using `pyo.Var()`, both constrained to be non-negative real numbers.
4. **Define the Objective Function:** Defines the objective function `obj` using `pyo.Objective()`. The `sense=pyo.maximize` indicates we are maximizing the function. 
5. **Define Constraints:** Defines the constraints using `pyo.Constraint()`:
    - `tank_capacity`: Represents the tank capacity constraint.
    - `stability1` and `stability2`: Represent the stability constraints. 
6. **Solve the Model:**
    - Creates a solver object using `pyo.SolverFactory('ipopt')`. You can choose a different solver appropriate for nonlinear problems.
    - Solves the model using `solver.solve(model)`.
7. **Print the Results:** Prints the solver status and the optimal values of the decision variables. 

This code provides a basic framework for solving the optimization problem. You can expand it by adding error handling, detailed result analysis, and potentially exploring different solvers for comparison.


## 4. Input problem data and try running the generated code

In [108]:
import pyomo.environ as pyo

# Create a concrete model
model = pyo.ConcreteModel()

# Define decision variables
model.x1 = pyo.Var(domain=pyo.NonNegativeReals)
model.x2 = pyo.Var(domain=pyo.NonNegativeReals)

# Define the objective function
model.obj = pyo.Objective(expr=pyo.log(model.x1 * model.x2) - (model.x1 + 2*model.x2)**(1/3), sense=pyo.maximize)

# Define constraints
model.tank_capacity = pyo.Constraint(expr=model.x1 + model.x2 <= 80)
model.stability1 = pyo.Constraint(expr=(model.x1 - (model.x1 + model.x2)/2)**2 <= 10)
model.stability2 = pyo.Constraint(expr=(model.x2 - (model.x1 + model.x2)/2)**2 <= 10)

# Solve the model
solver = pyo.SolverFactory('ipopt') # You can choose a different solver if you prefer
results = solver.solve(model)

# Print the results
print(results)
print("x1:", pyo.value(model.x1))
print("x2:", pyo.value(model.x2))


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 2
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.11.1\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.03985762596130371
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

x1: 43.162277233647174
x2: 36.8377229631074


## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [111]:
print(response.text)

## Mathematical Optimization Model

**Parameters:**

*  None

**Decision Variables:**

*  $x_1$: Litres of the first component used.
*  $x_2$: Litres of the second component used.

**Objective Function:**

Maximize $Z = \ln(x_1 \cdot x_2) - \sqrt[3]{x_1 + 2x_2}$ 

**Constraints:**

1. **Tank Capacity:** $x_1 + x_2 \leq 80$
2. **Stability:** $(x_1 - \frac{x_1 + x_2}{2})^2 \leq 10$
3. **Stability:** $(x_2 - \frac{x_1 + x_2}{2})^2 \leq 10$
4. **Non-negativity:** $x_1 \geq 0$
5. **Non-negativity:** $x_2 \geq 0$ 

**Explanation:**

* The objective function maximizes the quality of the end product based on the given formula.
* Constraint 1 ensures that the total volume of liquids used does not exceed the tank's capacity.
* Constraints 2 and 3 enforce the stability requirement by limiting the squared difference between each component's volume and the mean volume.
* Constraints 4 and 5 ensure that the volume of each component used is non-negative.

This model represents the problem mathematica

In [110]:
print(response2.text)

```python
import pyomo.environ as pyo

# Create a concrete model
model = pyo.ConcreteModel()

# Define decision variables
model.x1 = pyo.Var(domain=pyo.NonNegativeReals)
model.x2 = pyo.Var(domain=pyo.NonNegativeReals)

# Define the objective function
model.obj = pyo.Objective(expr=pyo.log(model.x1 * model.x2) - (model.x1 + 2*model.x2)**(1/3), sense=pyo.maximize)

# Define constraints
model.tank_capacity = pyo.Constraint(expr=model.x1 + model.x2 <= 80)
model.stability1 = pyo.Constraint(expr=(model.x1 - (model.x1 + model.x2)/2)**2 <= 10)
model.stability2 = pyo.Constraint(expr=(model.x2 - (model.x1 + model.x2)/2)**2 <= 10)

# Solve the model
solver = pyo.SolverFactory('ipopt') # You can choose a different solver if you prefer
results = solver.solve(model)

# Print the results
print(results)
print("x1:", pyo.value(model.x1))
print("x2:", pyo.value(model.x2))
```

This code defines the Pyomo model for the provided mathematical optimization problem. 

**Explanation:**

1. **Import Pyomo:** I